In [1]:
import numpy as np
import pandas as pd


# treliças


## Matriz de Rigidez **LOCAL**
Input:

* EA: rigidez, default é 1 
*  l: comprimento da barra
* teta_rad: Angulo da incidência em Radianos

Return:
*   k_local
*   T







In [2]:
def MATRIX_RIG_LOCAL (EA, l, teta_rad, GL, incidencia):

  
  Matrix_Rig_Template = np.array( [[1 ,0,-1,0],
                                   [0 ,0, 0,0],
                                   [-1,0, 1,0],
                                   [0 ,0, 0,0]])
  
  T = np.array( [[np.cos(teta_rad) ,np.sin(teta_rad),0,0],
                            [-np.sin(teta_rad) ,np.cos(teta_rad), 0,0],
                            [0,0, np.cos(teta_rad),np.sin(teta_rad)],
                            [0 ,0, -np.sin(teta_rad),np.cos(teta_rad)]])
  
  rows = 4
  cols = GL
  ones = [[incidencia[0]-1], [incidencia[1]-1], [incidencia[2]-1], [incidencia[3]-1]]

  INC = [ [ 0 for i in range(cols) ] for j in range(rows) ]

  for i,row in enumerate(ones):
      for col in row:
          INC[i][col] = 1
  
  
  return Matrix_Rig_Template *(EA/l) , T, np.array(INC)

## Matriz de Rigidez **GLOBAL**
Inputs:

* k_local: Matriz de Rigidez Local
* T: Matriz de Transposição

Return: 

* K_GLobal: Matriz de Rigidez no sistema global

In [3]:
def TRANS_LOCAL_GLOBAL (k_local, T):
  '''
  aqui entra a matriz de rigidez local
  e sai a matriz de rigidez global
  '''
  return T.transpose() @ k_local @ T

In [4]:
def INCIDENCIA (GL,incidencia):

  rows = 4
  cols = GL
  ones = [[incidencia[0]-1], [incidencia[1]-1], [incidencia[2]-1], [incidencia[3]-1]]

  INC = [ [ 0 for i in range(cols) ] for j in range(rows) ]

  for i,row in enumerate(ones):
      for col in row:
          INC[i][col] = 1


  return(np.array(INC))

In [5]:
def ESPALHAMENTO(k_global, Incidencia):
  return Incidencia.transpose() @ k_global  @ Incidencia

In [6]:
barra1, T1, I1 = MATRIX_RIG_LOCAL(1,1,np.deg2rad(0), 8, [3,4,1,2])
barra2, T2, I2 = MATRIX_RIG_LOCAL(1,1,np.deg2rad(60), 8, [5,6,1,2])
barra3, T3, I3 = MATRIX_RIG_LOCAL(1,1,np.deg2rad(-60), 8, [3,4,5,6])
barra4, T4, I4 = MATRIX_RIG_LOCAL(1,1,np.deg2rad(00), 8, [8,7,5,6])

In [7]:

K_G_1 =TRANS_LOCAL_GLOBAL(barra1, T1)
K_G_2 =TRANS_LOCAL_GLOBAL(barra2, T2)
K_G_3 =TRANS_LOCAL_GLOBAL(barra3, T3)
K_G_4 =TRANS_LOCAL_GLOBAL(barra4, T4)
K_G_2.shape

(4, 4)

In [8]:
K_1 = ESPALHAMENTO(K_G_1,I1)
K_2 = ESPALHAMENTO(K_G_2,I2)
K_3 = ESPALHAMENTO(K_G_3,I3)
K_4 = ESPALHAMENTO(K_G_4,I4)
K_1

array([[ 1.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [9]:
K = K_1 + K_2 + K_3 + K_4
display(pd.DataFrame(K))

,0,1,2,3,4,5,6,7
0,1.250000,0.433013,-1.000000,0.000000,-0.250000,-0.433013,0.0,0.0
1,0.433013,0.750000,0.000000,0.000000,-0.433013,-0.750000,0.0,0.0
2,-1.000000,0.000000,1.250000,-0.433013,-0.250000,0.433013,0.0,0.0
3,0.000000,0.000000,-0.433013,0.750000,0.433013,-0.750000,0.0,0.0
4,-0.250000,-0.433013,-0.250000,0.433013,1.500000,0.000000,0.0,-1.0
5,-0.433013,-0.750000,0.433013,-0.750000,0.000000,1.500000,0.0,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
7,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.0,1.0


# Exercicio de aula


In [10]:
Fa = np.array( [[1],[0]])
Kaa = np.array( [[12,2],
                 [2,4]                 
])

Kab = np.array( [[-15360,11520,0,0],
                 [11520,-8640,0,-40000],
                 [0,0,-15360,-11520],
                 [0,-40000,-11520,-8640]])



Ub = np.array([[0.04],[0],[0],[0]])

In [11]:
Ua = np.linalg.inv(Kaa) @ (Fa)
print(Ua)
print(np.linalg.inv(Kaa))


[[ 0.09090909]
 [-0.04545455]]
[[ 0.09090909 -0.04545455]
 [-0.04545455  0.27272727]]


In [12]:
U = np.concatenate((Ua,Ub), axis = 0)
U

array([[ 0.09090909],
       [-0.04545455],
       [ 0.04      ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])

In [13]:
barra6, T_6, INC_6 = MATRIX_RIG_LOCAL(120000, 5,-np.arctan(3/4), 8, [1,2,5,6])
print(barra6)
print(T_6)
print(INC_6)

print(barra6 @ T_6 @ INC_6 @ U)

[[ 24000.      0. -24000.      0.]
 [     0.      0.      0.      0.]
 [-24000.      0.  24000.      0.]
 [     0.      0.      0.      0.]]
[[ 0.8 -0.6  0.   0. ]
 [ 0.6  0.8  0.   0. ]
 [ 0.   0.   0.8 -0.6]
 [ 0.   0.   0.6  0.8]]
[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 6 is different from 8)

# Porticos

In [ ]:
def MATRIX_RIG_LOCAL_p (EA_p, EI_p, l_p, teta_rad_p, GL_p, incidencia_p):

  
  Matrix_Rig_Template_p = np.array( [[(EA_p/l_p),0 ,0,(-(EA_p/l_p)),0,0],
                                   [0 ,(12*EI_p/(l_p^3)), (6*EI_p/(l_p^2)),0,(-(12*EI_p/(l_p^3))),(6*EI_p/(l_p^2))],
                                   [0,(6*EI_p/(l_p^2)), (4*EI_p/(l_p)),0,-(6*EI_p/(l_p^2)),(2*EI_p/(l_p))],
                                   [-(EA_p/l_p),0 ,0,(EA_p/l_p),0,0],
                                   [0 ,(-(12*EI_p/(l_p^3))), (-(6*EI_p/(l_p^2))),0,(12*EI_p/(l_p^3)),(-(6*EI_p/(l_p^2)))],
                                   [0,(6*EI_p/(l_p^2)), (2*EI_p/(l_p)),0,(-(6*EI_p/(l_p^2))),(4*EI_p/(l_p))]
                                   ])


  T_p = np.array( [ [np.cos(teta_rad_p) ,np.sin(teta_rad_p),0,0,0,0],
                    [-np.sin(teta_rad_p) ,np.cos(teta_rad_p), 0,0,0,0],
                    [0,0,1,0,0,0],
                    [0,0,0, np.cos(teta_rad_p) ,np.sin(teta_rad_p),0],
                    [0,0,0,-np.sin(teta_rad_p) ,np.cos(teta_rad_p), 0],
                    [0,0,0,0,0,1]])

  rows = 6
  cols = GL_p
  ones = [[incidencia_p[0]-1], [incidencia_p[1]-1], [incidencia_p[2]-1], [incidencia_p[3]-1],[incidencia_p[4]-1],[incidencia_p[5]-1]]

  INC_p = [ [ 0 for i in range(cols) ] for j in range(rows) ]

  for i,row in enumerate(ones):
      for col in row:
          INC_p[i][col] = 1
  
  
  return Matrix_Rig_Template_p, T_p, np.array(INC_p)


def TRANS_LOCAL_GLOBAL_p (k_local_p, T_p):
  '''
  aqui entra a matriz de rigidez local
  e sai a matriz de rigidez global
  '''
  return T_p.transpose() @ k_local_p @ T_p

def ESPALHAMENTO(k_global_p, Incidencia_p):
  return Incidencia_p.transpose() @ k_global_p  @ Incidencia_p
  

In [ ]:
barra1, T1, I1 = MATRIX_RIG_LOCAL_p((50000000*0.02),(50000000*0.0001), 1, np.deg2rad(np.arctan(4/3)), 9, [5,6,7,1,2,3])
barra2, T2, I2 = MATRIX_RIG_LOCAL_p((50000000*0.02),(50000000*0.0001), 1, np.deg2rad(np.arctan(0)), 9, [1,2,3,4,8,9])

Global_1 = TRANS_LOCAL_GLOBAL_p(barra1,T1)
Global_2 = TRANS_LOCAL_GLOBAL_p(barra2,T2)

k1 = ESPALHAMENTO (Global_1, I1)
k2 = ESPALHAMENTO (Global_2, I2)

k = k1  + k2

display(pd.DataFrame(k))


,0,1,2,3,4,5,6,7,8
0,1.999746e+06,15696.082831,161.836482,-1000000.0,-999745.946846,-15696.082831,161.836482,0.0,0.0
1,1.569608e+04,60254.053154,1.309638,0.0,-15696.082831,-30254.053154,-9998.690362,-30000.0,10000.0
2,1.618365e+02,1.309638,40000.000000,0.0,-161.836482,9998.690362,10000.000000,-10000.0,10000.0
3,-1.000000e+06,0.000000,0.000000,1000000.0,0.000000,0.000000,0.000000,0.0,0.0
4,-9.997459e+05,-15696.082831,-161.836482,0.0,999745.946846,15696.082831,-161.836482,0.0,0.0
5,-1.569608e+04,-30254.053154,9998.690362,0.0,15696.082831,30254.053154,9998.690362,0.0,0.0
6,1.618365e+02,-9998.690362,10000.000000,0.0,-161.836482,9998.690362,20000.000000,0.0,0.0
7,0.000000e+00,-30000.000000,-10000.000000,0.0,0.000000,0.000000,0.000000,30000.0,-10000.0
8,0.000000e+00,10000.000000,10000.000000,0.0,0.000000,0.000000,0.000000,-10000.0,20000.0


,0,1,2,3,4,5
0,10000.0,0.0,0.0,-10000.0,0.0,0.0
1,0.0,300.0,100.0,0.0,-300.0,100.0
2,0.0,100.0,200.0,0.0,-100.0,100.0
3,-10000.0,0.0,0.0,10000.0,0.0,0.0
4,0.0,-300.0,-100.0,0.0,300.0,-100.0
5,0.0,100.0,100.0,0.0,-100.0,200.0


In [32]:
import numpy as np
import pandas as pd


class Stiffness_Matrix():
    """

    """

    def __init__(self, length, orientation_degrees , incidence_list, EA = 0, EI = 0):
        self.l =  length
        self.orientation = orientation_degrees
        self.incidence = incidence_list




In [58]:

class Portico(Stiffness_Matrix):
    """
    """
    def __init__(self, length, orientation_degrees , incidence_list, EA = 1, EI = 1, Deg_freedom = 6):
        # list of attributes
        Stiffness_Matrix.__init__(self, length, orientation_degrees , incidence_list, EA, EI)
        self.EA = EA
        self.EI = EI
        self.DF = Deg_freedom
        self.orientation = orientation_degrees
        self.Stiff_M = np.array( [[(EA/length),0 ,0,(-(EA/length)),0,0],
                                   [0 ,(12*EI/(length^3)), (6*EI/(length^2)),0,(-(12*EI/(length^3))),(6*EI/(length^2))],
                                   [0,(6*EI/(length^2)), (4*EI/(length)),0,-(6*EI/(length^2)),(2*EI/(length))],
                                   [-(EA/length),0 ,0,(EA/length),0,0],
                                   [0 ,(-(12*EI/(length^3))), (-(6*EI/(length^2))),0,(12*EI/(length^3)),(-(6*EI/(length^2)))],
                                   [0,(6*EI/(length^2)), (2*EI/(length)),0,(-(6*EI/(length^2))),(4*EI/(length))]
                                   ])

        self.local2global_T_M = np.array( 
                    [ [np.cos(np.deg2rad(orientation_degrees)) ,np.sin(np.deg2rad(orientation_degrees)),0,0,0,0],
                    [-np.sin(np.deg2rad(orientation_degrees)) ,np.cos(np.deg2rad(orientation_degrees)), 0,0,0,0],
                    [0,0,1,0,0,0],
                    [0,0,0, np.cos(np.deg2rad(orientation_degrees)) ,np.sin(np.deg2rad(orientation_degrees)),0],
                    [0,0,0,-np.sin(np.deg2rad(orientation_degrees)) ,np.cos(np.deg2rad(orientation_degrees)), 0],
                    [0,0,0,0,0,1] ]
                    )
        
        rows = 6
        cols = Deg_freedom
        ones = [[incidence_list[0]-1], [incidence_list[1]-1], [incidence_list[2]-1], [incidence_list[3]-1],[incidence_list[4]-1],[incidence_list[5]-1]]

        INC = [ [ 0 for i in range(cols) ] for j in range(rows) ]

        for i,row in enumerate(ones):
            for col in row:
                INC[i][col] = 1
        self.Incidency_M = np.array(INC)

        self.Stiff_M_Global = self.local2global_T_M.transpose() * self.Stiff_M * self.local2global_T_M
        self.Stiff_M_Global_

        # Incidencia.transpose() @ k_global  @ Incidencia
    # methods

    def updade_EA(self, new_EA):
        self.EA =  new_EA

    def updade_EI(self, new_EI):
        self.EI =  new_EI


class Truss(Stiffness_Matrix):
    """
    """
    def __init__(self, length, orientation_degrees , incidence_list, EA = 1, Deg_freedom = 4):

        # list of attributes
        Stiffness_Matrix.__init__(self, length, orientation_degrees , incidence_list, EA)
        self.EA = EA
        self.DF = Deg_freedom
        self.orientation = orientation_degrees
        self.Stiff_M = np.array( [[1 ,0,-1,0],
                                   [0 ,0, 0,0],
                                   [-1,0, 1,0],
                                   [0 ,0, 0,0]] ) * (EA/length)
        self.local2global_T_M = np.array( [[np.cos(np.deg2rad(orientation_degrees)) ,np.sin(np.deg2rad(orientation_degrees)),0,0],
                            [-np.sin(np.deg2rad(orientation_degrees)) ,np.cos(np.deg2rad(orientation_degrees)), 0,0],
                            [0,0, np.cos(np.deg2rad(orientation_degrees)),np.sin(np.deg2rad(orientation_degrees))],
                            [0,0, -np.sin(np.deg2rad(orientation_degrees)),np.cos(np.deg2rad(orientation_degrees))]])
    

        rows = 4
        cols = Deg_freedom
        ones = [[incidence_list[0]-1], [incidence_list[1]-1], [incidence_list[2]-1], [incidence_list[3]-1]]

        INC = [ [ 0 for i in range(cols) ] for j in range(rows) ]

        for i,row in enumerate(ones):
            for col in row:
                INC[i][col] = 1
        self.Incidency_M = np.array(INC)
        self.Stiff_M_Global = self.local2global_T_M.transpose() * self.Stiff_M * self.local2global_T_M

    def updade_EA(self, new_EA):
        self.EA = new_EA

In [66]:
p1 = Truss(5,90,[1,2,3,4])


matrix_rig_local = p1.Stiff_M
matrix_transf = p1.local2global_T_M
matrix_rig_global = matrix_transf.transpose() *  matrix_rig_local @ matrix_transf
matrix_rig_global

array([[7.49879891e-34, 1.22464680e-17, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 7.49879891e-34, 1.22464680e-17],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])